In [5]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [6]:
BATCH_SIZE = 64
EPOCHS = 50
INPUT_SIZE = 11
LAYER_SIZE = 10
LATENT_SIZE = 3

In [7]:
# Load the data from the file

saved = np.load("testing.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.1)])
test_data = torch.Tensor(saved[int(len(saved)*0.9):])
len(test_data)

10000

In [8]:
device = "cuda"

In [9]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.l2a = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, LAYER_SIZE)
        self.l4 = nn.Linear(LAYER_SIZE, INPUT_SIZE)
    
    # Run some data through the encoder
    def encode(self, x):
        out = F.relu(self.l1(x))
        # return the mu and the sigma
        return self.l2a(out), self.l2b(out)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        out = F.relu(self.l3(x))
        return torch.sigmoid(self.l4(out)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [10]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + KLD

In [12]:
why = VAE().to(device)
optimizer = optim.Adam(why.parameters(), lr=0.001)

In [16]:
def train(e_count):
    why.train()
    train_loss = 0
    for i in range(0, len(train_data), BATCH_SIZE):
        batch = train_data[i:i+BATCH_SIZE].to(device)
        why.zero_grad()
        recons, mu, sigma = why(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Epoch: {e_count}, Loss: {train_loss/len(train_data)}")        
        
    
    

In [17]:
# Lets use CPU
def test(e_count):
    why.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = why(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [18]:
for i in range(EPOCHS):
    train(i)
for i in range(EPOCHS):
    test(i)

Epoch: 0, Loss: 7.333362358093262
Epoch: 1, Loss: 7.03670824508667
Epoch: 2, Loss: 6.942564561462403
Epoch: 3, Loss: 6.9157506980895995
Epoch: 4, Loss: 6.90827720336914
Epoch: 5, Loss: 6.9063365982055664
Epoch: 6, Loss: 6.905120349884033
Epoch: 7, Loss: 6.904976037597656
Epoch: 8, Loss: 6.904421378326416
Epoch: 9, Loss: 6.903751925659179
Epoch: 10, Loss: 6.903700095367432
Epoch: 11, Loss: 6.903808680725097
Epoch: 12, Loss: 6.903047301483154
Epoch: 13, Loss: 6.902515979003907
Epoch: 14, Loss: 6.902414444732666
Epoch: 15, Loss: 6.901397582244873
Epoch: 16, Loss: 6.902827927398682
Epoch: 17, Loss: 6.902663004302979
Epoch: 18, Loss: 6.903002351379395
Epoch: 19, Loss: 6.902345877075195
Epoch: 20, Loss: 6.901821361541748
Epoch: 21, Loss: 6.9018935852050785
Epoch: 22, Loss: 6.902493190765381
Epoch: 23, Loss: 6.9018340644836425
Epoch: 24, Loss: 6.902389503479004
Epoch: 25, Loss: 6.90074647064209
Epoch: 26, Loss: 6.901525462341309
Epoch: 27, Loss: 6.902431446838379
Epoch: 28, Loss: 6.9015588897

In [ ]:
for i in range(EPOCHS):
    train(i)